# Training
Seems like we're ready for training!

Be aware of:
IP : 131.114.137.168

username : boscolo

pssw : unipi!2020$



stai attento a usare al max 2 GPU

In [1]:
import sys, os, datetime
sys.path.append('/home/andrea/AI/ispr_yolo/data')
sys.path.append('/home/andrea/AI/ispr_yolo/model')

from yolo import YOLOv3
from lossfunction import YoloLoss
from DataPreprocessing import create_dataset
import tensorflow as tf

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():

#### Creating the Dataset

In [2]:
    img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/train' + '/*.jpg'
    train_ds = create_dataset(img_dir, batch = 20)

In [3]:
    img_dir = '/home/andrea/AI/ispr_yolo/data/dataset_bdd/images/100k/val' + '/*.jpg'
    val_ds = create_dataset(img_dir, batch = 20)

#### Compiling the model

In [4]:
    yolo = YOLOv3(size = 1280, training = True)
    nadam = tf.keras.optimizers.Nadam(learning_rate=1e-03)
    yolo.compile(loss=[YoloLoss(10, 'dense'), YoloLoss(10, 'medium'), YoloLoss(10, 'coarse') ], optimizer=nadam)

#### Setting up TensorBoard and callbacks

In [5]:
    # Clear any logs from previous runs
    !rm -rf ./logs/ 

In [ ]:
    %load_ext tensorboard
    logdir = os.path.join("logs/fit", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    

In [7]:
    # Callbacks

    logging = TensorBoard(log_dir, histogram_freq=1)
    checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
                                 monitor='val_loss', save_weights_only=True,
                                 save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

In [8]:
    %reload_ext tensorboard
    %tensorboard --logdir logs

/bin/bash: line 0: kill: (20932) - No such process


### Training

In [ ]:
    yolo.fit(train_ds,
             epochs = 2,
             callbacks = callbacks=[logging, checkpoint],
             validation_data = val_ds)
    yolo.save_weights(log_dir + 'trained_weights.h5')

Epoch 1/2
    1/69863 [..............................] - ETA: 1:02:26 - loss: 59859.9219 - tf_op_layer_dense_grid_loss: 32032.7227 - tf_op_layer_medium_grid_loss: 8336.3975 - tf_op_layer_coarser_grid_loss: 19490.8008WARNING:tensorflow:From /home/andrea/anaconda2/envs/ai/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
   32/69863 [..............................] - ETA: 12:39:33 - loss: 26093.7031 - tf_op_layer_dense_grid_loss: 10922.0801 - tf_op_layer_medium_grid_loss: 5381.4053 - tf_op_layer_coarser_grid_loss: 9790.2158